In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
raw_data = pd.read_csv('netflix_titles.csv')
raw_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [2]:
data_split = raw_data.copy()
data_split = data_split.dropna(subset=['cast','country','date_added','rating','duration'],how='any')
data_split['director'].replace(np.nan,'Unknown',inplace=True)
# data_split.dropna(inplace=True)
data_split.shape

(7290, 12)

In [3]:
Movie_null = data_split[(data_split.director=='Unknown') & (data_split.type=='Movie')]
Movie_null.to_csv('Movie_null.csv')
TV_null = data_split[(data_split.director=='Unknown') & (data_split.type=='TV Show')]
TV_null.to_csv('TV_null.csv')
Movie = data_split[(data_split.type=='Movie')]
Movie.to_csv('Movie.csv')
TV = data_split[(data_split.type=='TV Show')]
TV.to_csv('TV.csv')

In [4]:
Movie_null.shape

(92, 12)

In [5]:
TV_null.shape

(1866, 12)

In [6]:
data_split['date_added']=pd.to_datetime(data_split['date_added'])
data_split['year_added']=data_split['date_added'].apply(lambda x: int(x.year))
data_split['month_added']=data_split['date_added'].apply(lambda x: int(x.month))
# data_split.head()

In [7]:
cast = pd.DataFrame(data_split['cast'].apply(lambda x: str(x).split(', ')).tolist(),index=data_split['show_id'])
cast=cast.stack().reset_index().drop('level_1',axis=1)
cast.rename(columns={0:'actor'},inplace=True)
# cast.head()

In [8]:
director = pd.DataFrame(data_split['director'].apply(lambda x: str(x).split(', ')).tolist(),index=data_split['show_id'])
director=director.stack().reset_index().drop('level_1',axis=1)
director.rename(columns={0:'director'},inplace=True)
# director.head()

In [9]:
country = pd.DataFrame(data_split['country'].apply(lambda x: str(x).split(', ')).tolist(),index=data_split['show_id'])
country=country.stack().reset_index().drop('level_1',axis=1)
country.rename(columns={0:'country'},inplace=True)
# country.head()

In [10]:
genre = pd.DataFrame(data_split['listed_in'].apply(lambda x: str(x).split(', ')).tolist(),index=data_split['show_id'])
genre=genre.stack().reset_index().drop('level_1',axis=1)
genre.rename(columns={0:'genre'},inplace=True)
# genre.head()

In [11]:
temp = cast.merge(director, on=['show_id'], how='inner')
temp1 = temp.merge(country, on=['show_id'], how='inner')
temp2 = temp1.merge(genre, on=['show_id'], how='inner')
# temp2.head()

In [12]:
data = temp2.merge(data_split[['show_id','type','title','date_added','release_year','rating','duration',
                             'description','year_added','month_added']],
                             on='show_id',how='inner').iloc[:,[0,5,6,1,2,3,4,7,8,9,10,11,12,13]]
# data.head()

In [13]:
movies = data[data['type']=='Movie']
tv_shows = data[data['type']=='TV Show']
tv=tv_shows['duration'].apply(lambda x:int(str(x).split(' ')[0]))
tv_shows=tv_shows.drop('duration',axis=1)
tv_shows['seasons']=tv
# tv_shows.head()

In [14]:
movie=movies['duration'].apply(lambda x:int(str(x).split(' ')[0]))
movies=movies.drop('duration',axis=1)
movies['duration']=movie
# movies.head()

In [15]:
t = movies[(movies['type']!='Unknown')&(movies['director']!='Unknown')].groupby(['type','director']).agg(
    {'show_id':'nunique'}).reset_index().sort_values(
    'show_id',ascending=False).head(80)
t1=movies.merge(t,on=['type','director'],how='inner').groupby(['type','director']).agg(
    {'show_id_x':'nunique'}).reset_index().sort_values('show_id_x',ascending=False).head(15).rename(
    {'show_id_x':'Total Movies'},axis=1)
t1

,type,director,Total Movies
18,Movie,Jan Suter,21
53,Movie,Raúl Campos,19
20,Movie,Jay Karas,15
38,Movie,Marcus Raboy,14
6,Movie,Cathy Garcia-Molina,13
39,Movie,Martin Scorsese,12
77,Movie,Youssef Chahine,12
19,Movie,Jay Chapman,12
68,Movie,Steven Spielberg,11
12,Movie,Don Michael Paul,10


In [16]:
# raw_data['date_added']=pd.to_datetime(raw_data['date_added'])
# raw_data['year_added']=raw_data['date_added'].apply(lambda x: int(x.year))
# raw_data['month_added']=raw_data['date_added'].apply(lambda x: int(x.month))
# raw_data.head()

In [17]:
raw_data.dropna(subset=['cast','country','date_added','rating','duration'],how='any')
raw_data.to_csv('data_netflix.csv')